In [ ]:
# set package parent path
import os
import sys

def get_parent_path(parent_folder_name: str = "src"):
    cur_path = os.path.abspath('')
    parent_path = cur_path[:cur_path.find(parent_folder_name) + len(parent_folder_name)]

    return parent_path   

parent_path = get_parent_path()
sys.path.append(parent_path)

In [ ]:
from transformers import (AutoConfig, AutoTokenizer,
                          DataCollatorForTokenClassification, IntervalStrategy,
                          Trainer, TrainingArguments)

from models.base.single_task import SRLModel
from data.preprocessing.base_model_dataset.singletask_dataset import \
    SingleTaskBioDataset
from data.preprocessing.task_dataset.srl_dataset import SrlBioDataset
from data.preprocessing.task_dataset.task_dataset import DatasetSize
from utils.metric.srl import compute_metrics
from utils.trainer import BaseModelArgs, setup_before_train
from utils.visualize import ModelVisualize

# Setup

In [ ]:
# ----------------------------------------------------------------------------------
srl_dataset = SrlBioDataset(
    model_name=args.encoder_model_name,
    label_file_name=args.label_file_path,
)

srl_dataset = srl_dataset.from_cache(args.data_file_path)

bio_dataset = SingleTaskBioDataset(task_dataset=srl_dataset)

bio_datadict_train = bio_dataset.get_train_datasets()
bio_datadict_val = bio_dataset.get_valid_datasets()


srl_label_vectorizer = srl_dataset.get_label_vectorizer(args.label_file_path)

label_vocab = srl_label_vectorizer.label_vocab

config = AutoConfig.from_pretrained(args.encoder_model_name, num_labels=len(label_vocab), id2label=label_vocab.rev_vocab, label2id=label_vocab.vocab)

tokenizer = AutoTokenizer.from_pretrained(args.encoder_model_name, padding=True)


model = SRLModel(config=config)


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

def compute_metrics_callback(p):
    return compute_metrics(p, label_vocab)

training_args = TrainingArguments(
    output_dir=args.model_save_dir,
    logging_dir=args.logging_dir,
    learning_rate=args.lr,
    per_device_train_batch_size=args.batch_size,
    per_device_eval_batch_size=args.batch_size,
    num_train_epochs=args.epochs,
    logging_steps=1000,
    save_steps=args.eval_steps,
    eval_steps=args.eval_steps,
    evaluation_strategy=IntervalStrategy.STEPS,
    push_to_hub=True,
    hub_private_repo=True,
    max_steps=4500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bio_datadict_train,
    eval_dataset=bio_datadict_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_callback,
)

# Train

In [ ]:
output = trainer.train()